In [2]:
import torch
import torch.nn.functional as Fun
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import KFold

In [4]:
def pre_data(path1,img_prd_path):
    df = pd.read_excel(path1)
    df.drop_duplicates(subset=['住院号'], keep='first', inplace=True)
    df2 = pd.read_excel(img_prd_path)
    df = pd.merge(df,df2)
#     df.to_excel('../data/输入网络测试数据.xlsx',index=False)
    df_x = df[['t1','t2','p1','p2','时间差','img_pred']]
    df_y = df[['label']]
    X = np.array(df_x)
    Y = np.array(df_y)
    X_tensor = torch.tensor(np.array(X),dtype=torch.float32)
    Y_tensor = torch.tensor(np.array(Y)).squeeze(dim=1)
    return X_tensor, Y_tensor

In [5]:
def get_acc(outputs, labels):
    """计算acc"""
    _,predict = torch.max(outputs.data,1)
    total_num = labels.shape[0]*1.0
    correct_num = (labels == predict).sum().item()
    acc = correct_num / total_num
    label_0,label_1 = 0,0
    cre_0,cre_1 = 0,0
    for i,j in zip(predict,labels):
        if j == 0:
            label_0 += 1
            if i == j:
                cre_0 += 1
        if j == 1:
            label_1 += 1
            if i == j:
                cre_1 += 1
    pd.DataFrame({'data_pred':list(predict.numpy().tolist()),
                  'labels':list(labels.numpy().tolist())}).to_excel('../data/时间推算结果.xlsx',index=False)
#     df_save = pd.read_excel('../data/输入网络测试数据.xlsx')
#     df_save['data_pred'] = list(predict.numpy().tolist())
#     df_save.to_excel('../data/数据推算预测结果.xlsx',index=False)
    print(acc)
    print('总的准确率:%.4f\t (%d + %d)\t/%d'%((cre_0 + cre_1) / len(predict), cre_0, cre_1, len(predict)))
    print('标签为0的准确率：%.4f\t(%d/%d)'%((cre_0 / label_0),cre_0,label_0))
    print('标签为1的准确率：%.4f\t(%d/%d)'%((cre_1 / label_1),cre_1,label_1))
    return acc

#     return acc

In [6]:
def get_acc_test(outputs, labels):
    """计算acc"""
    _,predict = torch.max(outputs.data,1)
    total_num = labels.shape[0]*1.0
    correct_num = (labels == predict).sum().item()
    acc = correct_num / total_num
    label_0,label_1 = 0,0
    cre_0,cre_1 = 0,0
    for i,j in zip(predict,labels):
        if j == 0:
            label_0 += 1
            if i == j:
                cre_0 += 1
        if j == 1:
            label_1 += 1
            if i == j:
                cre_1 += 1
    df1 = pd.DataFrame({'data_pred':list(predict.numpy().tolist()),
                          'labels':list(labels.numpy().tolist())})
#     df1.to_excel('../data/时间推算结果.xlsx',index=False)
#     df_save = pd.read_excel('../data/输入网络测试数据.xlsx')
#     df_save['data_pred'] = list(df1['data_pred'])
#     df_save.to_excel('../data/数据推算预测结果.xlsx',index=False)
    print(acc)
    print('总的准确率:%.4f\t (%d + %d)\t/%d'%((cre_0 + cre_1) / len(predict), cre_0, cre_1, len(predict)))
    print('标签为0的准确率：%.4f\t(%d/%d)'%((cre_0 / label_0),cre_0,label_0))
    print('标签为1的准确率：%.4f\t(%d/%d)'%((cre_1 / label_1),cre_1,label_1))
    return acc


In [7]:
# 定义BP神经网络
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)
        self.out = torch.nn.Linear(n_hidden, n_output)
        
    def forward(self,x):
        x = Fun.relu(self.hidden(x))
        x = self.out(x)
        return x

In [10]:
def main():
    trainpath_data = '../data/图像输出结果/训练集——月经周期推算（唯一编号）一一对应.xlsx'
    trainpath_img ='../data/图像输出结果/训练图片网络输出结果2.xlsx'
    testpath_data = '../data/图像输出结果/测试集——月经周期推算（唯一编号）一一对应.xlsx'
    testpath_img ='../data/图像输出结果/测试集图片网络输出结果2.xlsx'
    
    X_trian,Y_trian = pre_data(trainpath_data,trainpath_img)
    X_test,Y_test = pre_data(testpath_data,testpath_img)
    
    
    acc_total = 0
    epoch = 3
    
    kf = KFold(n_splits=epoch,shuffle=True,random_state=33)
    for train_index,test_index in kf.split(X_trian):
        x1,y1 = X_trian[train_index],Y_trian[train_index]
        x2,y2 = X_trian[test_index],Y_trian[test_index]
#         x1,x2,y1,y2 = train_test_split(X_trian,Y_trian,test_size=0.33)
        net = Net(n_feature=6,n_hidden=20,n_output=2)
        optimizer = torch.optim.SGD(net.parameters(), lr = 0.05)
        loss_func = torch.nn.CrossEntropyLoss()
        
        for i in range(1000):
            out = net(x1)
            loss = loss_func(out, y1)
            # 输出与label对比
            optimizer.zero_grad()
            # 初始化
            loss.backward()
            optimizer.step()

        out = net(x2)
        acc_1 = get_acc(out,y2)
        acc_total += acc_1
        print('_'*10)
        
    print('交叉验证%.4f'%(acc_total/epoch))
    print('_'*20)
    out = net(X_test)
    get_acc_test(out,Y_test)

In [15]:
if __name__=="__main__":
    main()

0.7027027027027027
总的准确率:0.7027	 (51 + 53)	/148
标签为0的准确率：0.6986	(51/73)
标签为1的准确率：0.7067	(53/75)
__________
0.7346938775510204
总的准确率:0.7347	 (61 + 47)	/147
标签为0的准确率：0.8356	(61/73)
标签为1的准确率：0.6351	(47/74)
__________
0.782312925170068
总的准确率:0.7823	 (69 + 46)	/147
标签为0的准确率：0.8961	(69/77)
标签为1的准确率：0.6571	(46/70)
__________
交叉验证0.7399
____________________
0.7919463087248322
总的准确率:0.7919	 (72 + 46)	/149
标签为0的准确率：0.9000	(72/80)
标签为1的准确率：0.6667	(46/69)
